In [554]:
import numpy as np
import math
import sympy as sp
import matplotlib.pyplot as plt


def diff(f, var = 'x y', d = 'y'):
    d = sp.symbols(d)
    x, y = sp.symbols(var)
    
    f_sym = f(x, y)
    
    df = sp.diff(f_sym, d)
    
    df = sp.lambdify((x, y), df)
    return df

dy = lambda t, y: y - t**2 + 1  

a, b = 0, 2
y0 = 0.5
h = 0.2


**Método de Euler**

In [555]:
def euler(f, a, b, y0, n = 10):
    h = (b - a) / n
    t = a
    w = y0
    
    for i in range(1, n+1):
        w = w + h * f(t, w)
        t = a + i * h
    return w

w = euler(dy, a, b, y0)
print(w)

4.865784504320001


**Método de Taylor de Ordem Superior**

In [556]:
def taylor(d, a, b, y0, n = 10):
    h = (b - a) / n
    X = np.arange(a, b, h)
    Y = [y0]
    
    order = len(d) + 1
    for j in range(len(X) - 1):
        taylor_sum = Y[j]
        for i in range(1,order):
            taylor_sum += (d[i-1](X[j], Y[j]) * h**i / math.factorial(i))
        Y.append(taylor_sum)
    return Y[-1]

d = [dy,  diff(dy, var = 't y')]

solution = taylor(d, a, b, y0)
print(solution)

4.8441317887999995


**Método de Runge-Kutta**

In [557]:
def runge_kutta(f, a, b, y0, n = 10):
    h = (b - a) / n 
    t = a
    w = y0
    
    for i in range(n):
        K1 = h * f(t, w)
        K2 = h * f(t + h / 2, w + K1 / 2)
        K3 = h * f(t + h / 2, w + K2 / 2)
        K4 = h * f(t + h, w + K3)
        
        w = w + (K1 + 2 * K2 + 2 * K3 + K4) / 6
        t = t + h
    
    return w

solution = runge_kutta(dy, a, b, y0)
print(solution)


5.305363000692655


**Método de Runge-Kutta-Fehlberg**

In [558]:
def runge_kutta_fehlberg(f, a, b, y0, e = 1e-5, n = 10):
    h = (b-a) / n
    t = a
    w = y0
    
    while t < b:
        if t + h > b:
            h = b - t
        
        K1 = h * f(t, w)
        K2 = h * f(t + (1/4) * h, w + (1/4) * K1)
        K3 = h * f(t + (3/8) * h, w + (3/32) * K1 + (9/32) * K2)
        K4 = h * f(t + (12/13) * h, w + (1932/2197) * K1 - (7209/2197) * K2 + (7296/2197) * K3)
        K5 = h * f(t + h, w + (439/216) * K1 - 8 * K2 + (3680/513) * K3 - (845/4104) * K4)
        K6 = h * f(t + (1/2) * h, w - (8/27) * K1 + 2 * K2 - (3544/2565) * K3 + (1859/4104) * K4 - (11/40) * K5)
        
        R = abs( (1/360) * K1 - (128/4275) * K3 - (2197/75240) * K4 + (1/50) * K5 + (2/55) * K6 ) / h
        
        if R <= e:
            t = t + h
            w = w + 25/216 * K1 + 1408/2565 * K3 + 2197/4104 * K4 - 1/5 * K5
        
        delta = 0.84 * (e / R)**(1/4)
        h = h * min(max(delta, 0.1), 4.0)
    
    return w

solution = runge_kutta_fehlberg(dy, a, b, y0)

print(solution)


5.304781449490056


In [559]:
def runge_kutta4(f, a, y0, h):
    K1 = h * f(a, y0)
    K2 = h * f(a + h/2, y0 + K1/2)
    K3 = h * f(a + h/2, y0 + K2/2)
    K4 = h * f(a + h, y0 + K3)
    
    return y0 + (K1 + 2*K2 + 2*K3 + K4) / 6.0

**Método Explícito de Adams-Bashfort**

In [560]:
def predictor(f, t, y, h, i):
    fn = f(t[i], y[i])
    fn1 = f(t[i-1], y[i-1])
    fn2 = f(t[i-2], y[i-2])
    fn3 = f(t[i-3], y[i-3])

    return y[i] + (h / 24) * (55*fn - 59*fn1 + 37*fn2 - 9*fn3)


In [561]:
def adams_bashfort(f, a, b, y0, h):
    n = int((b - a) / h)
    
    t = np.zeros(n + 1)
    y = np.zeros(n + 1)

    t[0] = a
    y[0] = y0

    for i in range(3):
        y[i+1] = runge_kutta4(f, t[i], y[i], h)
        t[i+1] = t[i] + h

    for i in range(3, n):
        t[i+1] = t[i] + h
        y[i+1] = predictor(f, t, y, h, i)
    
    return y[-1]

solution = adams_bashfort(dy, a, b, y0, h)
print(solution)


5.307508181393281


**Método Implícito de Adams-Moulton**

In [562]:
def corrector(f, t, y, h, i):
    y_predict = predictor(f, t, y, h, i)

    fn = f(t[i+1], y_predict)
    fn1 = f(t[i], y[i])
    fn2 = f(t[i-1], y[i-1])
    fn3 = f(t[i-2], y[i-2])
    
    return y[i] + (h / 24) * (9*fn + 19*fn1 - 5*fn2 + fn3)


**Método Preditor-Corretor**

In [563]:
def predictor_corrector(f, a, b, y0, h):

    n = int((b - a) / h)
    
    t = np.zeros(n + 1)
    y = np.zeros(n + 1)

    t[0] = a
    y[0] = y0

    for i in range(3):
        y[i+1] = runge_kutta4(f, t[i], y[i], h)
        t[i+1] = t[i] + h

    for i in range(3, n):
        t[i+1] = t[i] + h
        y[i+1] = corrector(f, t, y, h, i)
    
    return y[-1]

solution = predictor_corrector(dy, a, b, y0, h)
print(solution)

5.3053706715158455
